## Overview

Link to my fork: https://github.com/JaymanR/llm_engineering

This is a chatbot that takes orders for a restaurant and stores the orders in a database. The bot can also answer questions regarding ingredients and price of the items. A receipt is also generated when the order is finalized.


In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3

In [ ]:
load_dotenv()

open_ai_api_key = os.getenv("OPENAI_API_KEY")
if open_ai_api_key:
    print(f"OpenAI API key loaded successfully.")
else:
    print("Key not found.")

MODEL = "gpt-4.1-mini"
openai = OpenAI()

DB = "burgerz.db"


In [ ]:
system_message = """
You are a helpful assistant for the restaurant Burgerz.
When a customer wants to place an order, call create_order() to start a new order.
Then use add_to_order() to add items to their order, providing the order ID returned from create_order().
You can help customers by describing menu items and getting prices.
When the customer is done ordering, use get_order() to show them their complete order with itemized receipt and total price(use new line per item for formatting the receipt when showing it to the customer).
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

Let's create three tables, one for the menu items, one for orders and a table for items in an order.

In [ ]:
with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute("""
                   CREATE TABLE IF NOT EXISTS menu (
                       item_name TEXT PRIMARY KEY,
                       price REAL,
                       description TEXT
                    )
                    """)
    cursor.execute("""
                   CREATE TABLE IF NOT EXISTS orders (
                      order_id INTEGER PRIMARY KEY AUTOINCREMENT,
                      created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                      total_price REAL DEFAULT 0
                   )
                    """)
    cursor.execute("""
                   CREATE TABLE IF NOT EXISTS order_items (
                       item_id INTEGER PRIMARY KEY AUTOINCREMENT,
                       order_id INTEGER NOT NULL,
                       item_name TEXT,
                       quantity INTEGER DEFAULT 1,
                       price_per_item REAL,
                       FOREIGN KEY (order_id) REFERENCES orders(order_id)
                   )
                   """)
    conn.commit()

In [ ]:
def get_item_price(item_name):
    print(f"DATABASE TOOL CALLED: Getting price for {item_name}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT price FROM menu WHERE item_name = ?", (item_name.lower(),))
        result = cursor.fetchone()
        return f"Price for {item_name} is ${result[0]}" if result else f"Sorry, {item_name} is not on the menu."

In [ ]:
def get_item_description(item_name):
    print(f"DATABASE TOOL CALLED: Getting description for {item_name}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT description FROM menu WHERE item_name = ?", (item_name.lower(),))
        result = cursor.fetchone()
        return f"{item_name}: {result[0]}" if result else f"Sorry, {item_name} is not on the menu."

In [ ]:
def get_order(order_id):
    print(f"DATABASE TOOL CALLED: Getting order details for order ID {order_id}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("""
                       SELECT created_at FROM orders WHERE order_id = ?
                       """, (order_id,))
        order_info = cursor.fetchone()
        if not order_info:
            return f"No order found with ID {order_id}."
        created_at = order_info[0]
        
        cursor.execute("""
                       SELECT item_name, quantity, price_per_item
                       FROM order_items
                       WHERE order_id = ?
                       """, (order_id,))
        items = cursor.fetchall()
        if not items:
            return f"Order #{order_id} (Date: {created_at}) has no items."
        total_price = sum(quantity * price_per_item for _, quantity, price_per_item in items)
        item_details = "\n".join([f"{item_name} x{quantity} @ ${price_per_item:.2f} each" for item_name, quantity, price_per_item in items])
        return f"Order #{order_id}\nDate: {created_at}\n{item_details}\nTotal Price: ${total_price:.2f}"

In [ ]:
def create_order():
    print(f"DATABASE TOOL CALLED: Creating a new order", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("INSERT INTO orders DEFAULT VALUES")
        conn.commit()
        order_id = cursor.lastrowid
        return f"New order created with Order ID: {order_id}."

In [ ]:
def add_to_order(order_id, item_name, quantity):
    print(f"DATABASE TOOL CALLED: Attempting to add {quantity} of {item_name} to order ID {order_id}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT price FROM menu WHERE item_name = ?", (item_name.lower(),))
        result = cursor.fetchone()
        if not result:
            return f"Sorry, {item_name} is not on the menu."
        price_per_item = result[0]
        cursor.execute("""
                       INSERT INTO order_items (order_id, item_name, quantity, price_per_item)
                       VALUES (?, ?, ?, ?)
                       """, (order_id, item_name.lower(), quantity, price_per_item))
        conn.commit()
        return f"Added {quantity} of {item_name} to order ID {order_id}."

In [ ]:
def set_menu_item(item_name, price, description):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("""
                       INSERT INTO menu (item_name, price, description)
                       VALUES (?, ?, ?)
                       ON CONFLICT(item_name) DO UPDATE SET
                          price=excluded.price,
                          description=excluded.description
                       """, (item_name.lower(), price, description))
        conn.commit()

In [ ]:
menu_items = {
    "veggie burger": (
        7.99,
        """House-made veggie patty (black beans, quinoa, and roasted vegetables)
        topped with crisp lettuce, ripe tomato, melted cheddar, and a zesty garlic aioli
        on a toasted whole-wheat bun; served with your choice of side.""",
    ),
    "chicken burger": (
        8.99,
        """Marinated and grilled chicken breast with smoked paprika aioli, crisp lettuce, tomato, pickles,
        and melted Swiss cheese on a buttery brioche bun; served hot.""",
    ),
    "turkey burger": (
        8.99,
        """Lean herb-seasoned turkey patty topped with Swiss cheese, peppery arugula, tomato,
        and a cranberry-mayo spread on a toasted multigrain bun.""",
    ),
    "beef burger": (
        8.99,
        """100% Angus beef patty seared to order, topped with American cheese,
        caramelized onions, crisp lettuce, tomato, and house sauce on a toasted sesame bun.""",
    ),
    "fries": (
        4.99,
        """Hand-cut russet potatoes, double-fried for extra crispness and seasoned with sea salt;
        served with ketchup and optional garlic-parmesan or spicy aioli dipping sauces.""",
    ),
    "soda": (
        3.99,
        """Ice-cold carbonated beverage available in cola, lemon-lime, root beer, or orange;
        served over ice with an optional lemon wedge.""",
    ),
}
for item_name, (price, description) in menu_items.items():
    set_menu_item(item_name, price, description)

In [ ]:
get_item_price("fries")

In [ ]:
get_item_description("fries")

In [ ]:
price_function = {
    "name": "get_item_price",
    "description": "Get the price of a menu item",
    "parameters": {
        "type": "object",
        "properties": {
            "item_name": {
                "type": "string",
                "description": "The name of the menu item to get the price for"
            },
        },
        "required": ["item_name"],
        "additionalProperties": False
    }
}

In [ ]:
description_function = {
    "name": "get_item_description",
    "description": "Get the description of a menu item",
    "parameters": {
        "type": "object",
        "properties": {
            "item_name": {
                "type": "string",
                "description": "The name of the menu item to get the description for"
            },
        },
        "required": ["item_name"],
        "additionalProperties": False
    }
}

In [ ]:
create_order_function = {
    "name": "create_order",
    "description": "Create a new order. Call this when the customer starts a new order.",
    "parameters": {
        "type": "object",
        "properties": {},
        "required": [],
        "additionalProperties": False
    }
}


In [ ]:
add_to_order_function = {
    "name": "add_to_order",
    "description": "Add an item to an existing order",
    "parameters": {
        "type": "object",
        "properties": {
            "order_id": {
                "type": "integer",
                "description": "The order ID to add the item to"
            },
            "item_name": {
                "type": "string",
                "description": "The name of the menu item to add"
            },
            "quantity": {
                "type": "integer",
                "description": "The quantity of the item to add",
                "default": 1
            }
        },
        "required": ["order_id", "item_name", "quantity"],
        "additionalProperties": False
    }
}


In [ ]:
get_order_function = {
    "name": "get_order",
    "description": "Get the details of an order, including all items and total price",
    "parameters": {
        "type": "object",
        "properties": {
            "order_id": {
                "type": "integer",
                "description": "The order ID to retrieve"
            }
        },
        "required": ["order_id"],
        "additionalProperties": False
    }
}


In [ ]:
tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": description_function},
    {"type": "function", "function": create_order_function},
    {"type": "function", "function": add_to_order_function},
    {"type": "function", "function": get_order_function},
]
tools

In [ ]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_item_price":
            arguments = json.loads(tool_call.function.arguments)
            item = arguments.get("item_name")
            price_details = get_item_price(item)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "get_item_description":
            arguments = json.loads(tool_call.function.arguments)
            item = arguments.get("item_name")
            description_details = get_item_description(item)
            responses.append({
                "role": "tool",
                "content": description_details,
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "create_order":
            order_result = create_order()
            responses.append({
                "role": "tool",
                "content": order_result,
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "add_to_order":
            arguments = json.loads(tool_call.function.arguments)
            order_id = arguments.get("order_id")
            item_name = arguments.get("item_name")
            quantity = arguments.get("quantity", 1)
            add_result = add_to_order(order_id, item_name, quantity)
            responses.append({
                "role": "tool",
                "content": add_result,
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "get_order":
            arguments = json.loads(tool_call.function.arguments)
            order_id = arguments.get("order_id")
            order_result = get_order(order_id)
            responses.append({
                "role": "tool",
                "content": order_result,
                "tool_call_id": tool_call.id
            })
    return responses

In [ ]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    return response.choices[0].message.content


    

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()